In [ ]:
# Restart the session afther this cell to avoid Google Colab errors
!pip install --upgrade --force-reinstall numpy==1.26.4 pandas

In [ ]:
!pip install pybibx
!pip install tabulate tqdm

In [ ]:
# Dowload .bib file
#!wget https://github.com/Valdecy/pyBibX/raw/main/assets/bibs/scopus.bib

In [ ]:
# Required Libraries
import textwrap

from pybibx.base import pbx_probe
from tabulate import tabulate

In [ ]:
# Load .bib
# Arguments: file_bib = 'filename.bib'; db = 'scopus', 'wos', 'pubmed'; del_duplicated = True, False
file_name = 'dados/scopus.bib'
database  = 'scopus'
bibfile   = pbx_probe(file_bib = file_name, db = database, del_duplicated = True)

In [ ]:
print(bibfile.data.document_type.value_counts())
filtro = ['Article','Conference paper']
bibfile.data = bibfile.data[bibfile.data['document_type'].isin(filtro)]
print(bibfile.data.document_type.value_counts())

In [ ]:
file_name_acm = 'dados/acm.bib'
database_acm  = 'acm'
bibfile_acm   = pbx_probe(file_bib = file_name_acm, db = database_acm, del_duplicated = True)


In [ ]:
bibfile.merge_database(file_bib=file_name_acm, db=database_acm, del_duplicated=True)

In [ ]:
# Health Analysis
health = bibfile.health_bib()

# Check Health
health

In [ ]:
print(bibfile.data['abstract'].head(2))

In [ ]:
print(bibfile.data.columns)

In [ ]:
!pip install pybtex
!pip install bibtexparser

In [ ]:
import pandas as pd
import bibtexparser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase


In [23]:
import time
import os
import random
from multiprocessing import Pool, cpu_count
from functools import partial
import pandas as pd
import ollama

# === CONFIGURAÇÕES GLOBAIS ===
MAX_REQUESTS_PER_MINUTE = 300
SECONDS_BETWEEN_REQUESTS = 60 / MAX_REQUESTS_PER_MINUTE
BATCH_SIZE = 1
WORKERS = min(8, cpu_count())
MODELS = ["llama3:8b", "gemma3:27b-it-qat", "phi4"]
TEMPERATURE = 0

QUERY = (
    #"Does this abstract discuss artificial intelligence in feedback for learning management systems or online learning environment on education?"
    "Analyze the following scientific article abstract and determine whether it "
     "addresses the use of artificial intelligence to provide feedback in virtual learning environments.\n"
     "Consider aspects such as: the application of AI techniques, automated feedback systems, "
     "digital educational platforms, and online learning. Respond only with ‘yes’ if the article is related, "
     "or ‘no’ if it is not.\n\n"
)

# === UTILITÁRIOS ===
def log(text: str, log_path: str) -> None:
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"{timestamp} - {text}\n")
    print(f"{timestamp} - {text}")


def chunk_dataframe(df, batch_size: int):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i : i + batch_size], i


# === CHAMADA AO LLM LOCAL ===
def call_local_llm(messages, model: str, temperature: float):
    response = ollama.chat(
        model=model,
        messages=messages,
        options={"temperature": temperature},
        stream=False
    )
    return response.message.content.strip()


# === PROCESSAMENTO DE LOTE COM RETRIES ===
def process_batch_with_retry(
    batch_df, global_index, query, model, temperature,
    seconds_between_requests, log_path, retry_limit=5
):
    retry_count = 0
    delay = seconds_between_requests + random.uniform(0, 5)

    while retry_count < retry_limit:
        try:
            if model == "cogito:8b":
                time.sleep(delay)
                messages = [
                    {"role": "system", "content": (
                        "Enable deep thinking subroutine."
                    )}
                ]
                prompt = f"{query}\n\nYou are a research assistant who helps analyze scientific articles. Restrict yourself to answering the question with exclusively 'yes' or 'no'.\n\n"
            else:
                time.sleep(delay)
                messages = [
                    {"role": "system", "content": (
                        "You are a research assistant who helps analyze scientific articles."
                    )}
                ]
                prompt = f"{query}\n\nRestrict yourself to answering the question with exclusively 'yes' or 'no'.\n\n"

            for i, row in batch_df.iterrows():
                prompt += f"Abstract {i + 1}:\n{row['abstract']}\n\n"

            messages.append({"role": "user", "content": prompt})

            content = call_local_llm(messages, model=model, temperature=temperature)
            answers = content.splitlines()

            results = []
            coluna = "relevant_" + model.split(":")[0]
            for answer, (_, row) in zip(answers, batch_df.iterrows()):
                result = row.to_dict()
                clean = answer.strip().lower()
                result[coluna] = (clean == "yes")
                results.append(result)

            log(f"[{model}] Lote {global_index} OK", log_path)
            return results

        except Exception as e:
            retry_count += 1
            wait_time = 2 ** retry_count + random.uniform(0, 1)
            log(f"[{model}][ERRO] Lote {global_index}, tentativa {retry_count}: {e}", log_path)
            time.sleep(wait_time)

    log(f"[{model}][FALHA] Lote {global_index} excedeu tentativas", log_path)
    return []


def process_args_wrapper(args, query, model, temperature, seconds_between_requests, log_path):
    return process_batch_with_retry(*args, query=query, model=model,
                                    temperature=temperature,
                                    seconds_between_requests=seconds_between_requests,
                                    log_path=log_path)


# === PIPELINE PARA UM MODELO ===
def analyze_abstracts_parallel(
    df: pd.DataFrame,
    query: str,
    model: str,
    batch_size: int,
    workers: int,
    result_csv_path: str,
    log_path: str,
    temperature: float,
    seconds_between_requests: float,
) -> pd.DataFrame:
    if os.path.exists(result_csv_path):
        acumulado = pd.read_csv(result_csv_path)
        start = len(acumulado)
        log(f"[{model}] Retomando do índice {start}", log_path)
    else:
        acumulado = pd.DataFrame()
        start = 0

    to_process = df.iloc[start:].reset_index(drop=True)
    batches = [
        (batch, idx + start)
        for batch, idx in chunk_dataframe(to_process, batch_size)
    ]

    log(f"[{model}] Iniciando {len(batches)} lotes com {workers} workers", log_path)

    with Pool(processes=workers) as pool:
        processor = partial(
            process_args_wrapper,
            query=query,
            model=model,
            temperature=temperature,
            seconds_between_requests=seconds_between_requests,
            log_path=log_path
        )
        for outcome in pool.imap_unordered(processor, batches):
            if outcome:
                df_part = pd.DataFrame(outcome)
                acumulado = pd.concat([acumulado, df_part], ignore_index=True)
                acumulado.to_csv(result_csv_path, index=False)

    log(f"[{model}] Processamento completo.", log_path)
    return acumulado


# === RUN_ALL_MODELS MODIFICADA ===
def run_all_models(df: pd.DataFrame) -> pd.DataFrame:
    #
    #Executa o pipeline para todos os modelos em MODELS e retorna um DataFrame combinado
    #contendo todas as colunas relevant_<model>.
    #
    combined = df.copy()

    for model in MODELS:
        model_name = model.split(":")[0]
        result_path = f"temp_files/resultados_parciais_{model_name}.csv"
        log_path = f"temp_files/log_execucao_{model_name}.txt"

                # ajusta workers para alguns modelos (exemplo)
        extra = 0
        if model in ("gemma3:27b-it-qat","phi4-mini"): extra = 4
        workers = WORKERS + extra

        resultados = analyze_abstracts_parallel(
            df=combined,
            query=QUERY,
            model=model,
            batch_size=BATCH_SIZE,
            workers=workers,
            result_csv_path=result_path,
            log_path=log_path,
            temperature=TEMPERATURE,
            seconds_between_requests=SECONDS_BETWEEN_REQUESTS
        )

        col = f"relevant_{model_name}"
        combined = combined.merge(
            resultados[[col]],
            left_index=True, right_index=True
        )

    return combined


In [ ]:
# import os
# import logging
# import asyncio
# import nest_asyncio
# import pandas as pd
# import ollama
# from tenacity import retry, wait_random_exponential, stop_after_attempt
# from langchain.llms.base import LLM
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.cache import InMemoryCache

# # === Logging Setup ===
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     datefmt="%Y-%m-%d %H:%M:%S"
# )
# logger = logging.getLogger(__name__)

# # === Enable nested asyncio (e.g., notebooks) ===
# nest_asyncio.apply()

# # === Configuration ===
# MODELS = ["llama3:8b", "gemma3:27b-it-qat", "phi4"]
# TEMPERATURE = 0
# cache = InMemoryCache()

# # === Prompt Template (yes/no only) ===
# prompt_template = PromptTemplate(
#     input_variables=["abstract"],
#     template=(
#         "You are a research assistant. Does this abstract discuss artificial intelligence in feedback for learning management systems on education? .\n"
#         "Respond with exactly 'yes' or 'no' (lowercase), and nothing else.\n\n"
#         "Abstract:\n{abstract}"
#     )
# )

# # === Ollama LLM Wrapper with Retry and Caching ===
# class OllamaLLM(LLM):
#     model_name: str
#     temperature: float

#     def __init__(self, model_name: str, temperature: float = 0):
#         super().__init__(model_name=model_name, temperature=temperature)
#         self.cache = cache

#     @property
#     def _llm_type(self) -> str:
#         return "ollama"

#     @retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(5))
#     def _call(self, prompt: str, stop=None) -> str:
#         response = ollama.chat(
#             model=self.model_name,
#             messages=[{"role": "user", "content": prompt}],
#             options={"temperature": self.temperature},
#             stream=False
#         )
#         return response.message.content.strip()

#     async def _acall(self, prompt: str, stop=None) -> str:
#         # Use synchronous call in async context
#         return self._call(prompt, stop)

# # === Pre-build Chains ===
# model_keys = [m.split(':')[0] for m in MODELS]
# chains = {
#     key: LLMChain(
#         llm=OllamaLLM(model_name=model, temperature=TEMPERATURE),
#         prompt=prompt_template
#     )
#     for key, model in zip(model_keys, MODELS)
# }

# # === Main Pipeline ===
# def run_all_models(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Evaluate each abstract concurrently for all models, returning a DataFrame
#     with additional columns relevant_<model_key> as booleans.
#     """
#     logger.info(f"Starting analysis of {len(df)} abstracts across models: {model_keys}")

#     async def analyze_all() -> dict[str, list[bool]]:
#         # Schedule one prediction per abstract per model
#         tasks = {
#             key: [chain.apredict(abstract=abs_text) for abs_text in df['abstract']]
#             for key, chain in chains.items()
#         }
#         # Gather all responses
#         raw_results = {
#             key: await asyncio.gather(*prompts)
#             for key, prompts in tasks.items()
#         }
#         # Parse responses to booleans
#         bool_results: dict[str, list[bool]] = {}
#         for key, raws in raw_results.items():
#             bools: list[bool] = []
#             for raw in raws:
#                 txt = raw.strip().lower()
#                 if txt == 'yes':
#                     bools.append(True)
#                 elif txt == 'no':
#                     bools.append(False)
#                 else:
#                     logger.warning(f"Unexpected response for model {key}: '{raw}' — defaulting to False")
#                     bools.append(False)
#             bool_results[key] = bools
#         return bool_results

#     # Execute the async pipeline
#     results = asyncio.run(analyze_all())
#     logger.info("Analysis completed. Building DataFrame.")

#     # Build output DataFrame
#     data = df.to_dict(orient='list')
#     for key, vals in results.items():
#         data[f'relevant_{key}'] = vals

#     return pd.DataFrame(data)


In [ ]:
def load_and_filter_bases(directory: str) -> pd.DataFrame:
    print(f"Carregando bases de '{directory}'")
    dfs = []
    for fname in sorted(os.listdir(directory)):
        if fname.lower().endswith('.csv'):
            path = os.path.join(directory, fname)
            try:
                df = pd.read_csv(path)
                print(f"{fname}: {len(df)} registros")
                dfs.append(df)
            except Exception as e:
                print(f"Erro lendo {fname}: {e}")
    if not dfs:
        return pd.DataFrame()
    df = pd.concat(dfs, ignore_index=True)
    return df


In [ ]:
dados = bibfile.data
df_ieee = load_and_filter_bases("dados")
colunas_desejadas_ieee = ['Document Title', 'Abstract', 'Author Affiliations', 'Authors', 'DOI', 'ISBNs',
                             'ISSN', 'Publication Title', 'Publication Year']
df_ieee = df_ieee[colunas_desejadas_ieee].copy()
#print(df_ieee.columns)
colunas_desejadas_scopus = ['title', 'abstract', 'journal', 
                            'affiliation', 'author', 'doi', 'isbn',
                             'issn', 'year']

df_scopus = dados[colunas_desejadas_scopus].copy()
df_scopus = df_scopus.rename(columns={
    'title': 'Document Title',
    'abstract': 'Abstract',
    'abbrev_source_title': 'Publication Title',
    'affiliation': 'Author Affiliations',
    'author': 'Authors',
    'doi': 'DOI',
    'isbn': 'ISBNs',
    'issn': 'ISSN',
    'journal': 'Publication Title',
    'references': 'References',
    'url': 'URL',
    'year': 'Publication Year'
})


# Certifique-se de que o DataFrame `dados` contém pelo menos as colunas 'abstract' e outras desejadas
#resultados = analyze_abstracts_parallel(dados, query=query_global, model=model, batch_size=5, workers=8)

print(df_ieee.shape, '\t', df_scopus.shape)
dados = pd.concat([df_ieee, df_scopus], ignore_index=True)
print('Antes da remoção de duplicados: ', dados.shape)
dados.columns = dados.columns.str.lower()
dados = dados.dropna(subset=['abstract'])
dados = dados.drop_duplicates(subset=['abstract'])
dados = dados.reset_index(drop=True)
print('Após remoção de duplicados: ', dados.shape)

resultados = run_all_models(dados)

# Salvar CSV final (opcional)
resultados.to_csv("temp_files/resultados_preliminares.csv", index=False)

Carregando bases de 'dados'
export_ieee_2015-2018.csv: 799 registros
export_ieee_2018-2022.csv: 695 registros
export_ieee_2022-2025.csv: 936 registros
(2430, 9) 	 (13595, 9)
Antes da remoção de duplicados:  (16025, 9)
Após remoção de duplicados:  (15614, 9)
2025-04-24 13:26:33 - [llama3:8b] Iniciando 15614 lotes com 8 workers


2025-04-24 13:26:33 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-04-24 13:26:33 - [llama3:8b] Lote 1 OK


2025-04-24 13:26:34 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-04-24 13:26:34 - [llama3:8b] Lote 8 OK


...

2025-04-24 13:30:35 - [llama3:8b] Lote 759 OK


2025-04-24 13:30:35 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-04-24 13:30:35 - [llama3:8b] Lote 763 OK


2025-04-24 13:30:35 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


2025-04-24 13:30:35 - [llama3:8b] Lote 771 OK


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
print(resultados.columns)
# Exibir os resultados
for model in MODELS:
    model_name = model.split(":")[0]
    print(f"\nResultados para o modelo {model_name}:")
    print(resultados[f'relevant_{model_name}'].value_counts())
#dados_filtered = resultados[resultados['relevant'] != 'False']
#dados_filtered.head(3)

## Avaliação da diferença entre os modelos

In [ ]:
def apply_committee(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adiciona ao DataFrame uma coluna 'relevant' que será 'true' se a
    maioria (≥3) das colunas ['a','b','c','d','e'] for 'true', caso
    contrário 'false'.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame contendo as colunas 'a','b','c','d' e 'e', com valores
        'true' ou 'false' (strings).

    Retorna
    -------
    pd.DataFrame
        O mesmo DataFrame, com a coluna 'relevant' adicionada.
    """
    cols = ['relevant_llama3', 'relevant_gemma3', 'relevant_phi4']
    # Verifica se as colunas existem no DataFrame
    for col in cols:
        if col not in df.columns:
            raise ValueError(f"Coluna {col} não encontrada no DataFrame.")
    # Conta quantos 'true' por linha
    true_counts = df[cols].eq(True).sum(axis=1)
    # Define 'relevant' = 'true' se true_counts >= 3, senão 'false'
    df['relevant'] = (true_counts >= 2).map({True: True, False: False})
    return df

In [ ]:
resultados_com_relevancia = apply_committee(resultados)
resultados_com_relevancia.to_csv("temp_files/resultados_finais_com_relevancia.csv", index=False)
print(resultados_com_relevancia['relevant'].value_counts())

In [ ]:
print(resultados_com_relevancia['relevant'].value_counts())
print(resultados_com_relevancia[resultados_com_relevancia['relevant'] == True])

## Critrérios de exclusão

1. Livro
2. Artigos resumidos
3. Revisões de literatura
4. Relatórios técnicos
5. Escrito em língua estrangeira que não seja o inglês
6. Simulou cenários


In [ ]:
# ----------------------------
# Critérios de Exclusão
# ----------------------------
criterios_exclusao: dict[str, str] = {
"QE1": "Is it a Book?",
"QE2": "Is it a Summarized articles",
"QE3": "Is it a Literature reviews",
"QE4": "Is it aTechnical reports",
"QE5": "Was it written in a language other than English",
"QE6": "Is it a Simulated scenarios"
}

In [ ]:
def verifica_criterio_llm(abstract: str, descricao: str, model: str, temperature: float) -> str:
    prompt = (
        f"The following exclusion criterion applies to the abstract below.?\n\n"
        f"Criterion: {descricao}\n\n"
        f"Abstract:\n{abstract}\n\n"
        "Answer only with 'yes' or 'no'."
    )
    # mesma função de chamada usada acima
    system_msg = {"role": "system", "content": "You are a research assistant."}
    user_msg   = {"role": "user",   "content": prompt}
    try:
        resp = call_local_llm([system_msg, user_msg], model=model, temperature=temperature)
        resp_clean = resp.strip().lower()
        return resp_clean if resp_clean in ("yes","no") else "não"
    except:
        return "no"

def apply_exclusion_criteria(
    df: pd.DataFrame,
    criterios: dict[str,str],
    model: str,
    temperature: float,
    log_path: str
) -> pd.DataFrame:
    for code, desc in criterios.items():
        log(f"Iniciando verificação do {code}: {desc}", log_path)
        df[code] = df["abstract"].apply(
            lambda txt: verifica_criterio_llm(txt, desc, model, temperature)
        )
    log("Verificação de todos os critérios concluída.", log_path)
    return df

In [ ]:
exclusion_log = "temp_files/log_exclusao.txt"
resultados_com_relevancia_apenas_True = resultados_com_relevancia[resultados_com_relevancia['relevant'] == True]
resultados_com_relevancia_apenas_True = resultados_com_relevancia_apenas_True.reset_index(drop=True)

papers_criterios_exclusao = apply_exclusion_criteria(
    resultados_com_relevancia_apenas_True,
    criterios_exclusao,
    model=MODELS[0],
    temperature=TEMPERATURE,
    log_path=exclusion_log
)

papers_criterios_exclusao.to_csv("temp_files/resultados_criterios_exclusao.csv", index=False)
print("Pipeline completa! Resultados em temp_files/resultados_criterios_exclusao.csv")

In [ ]:
def apply_committee_criterios_exclusao(df: pd.DataFrame, criterios: dict[str, str]) -> pd.DataFrame:
    """
    Adiciona ao DataFrame uma coluna 'incluso' que será 'true' se a
    maioria (≥3) das colunas ['a','b','c','d','e'] for 'true', caso
    contrário 'false'.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame contendo as colunas , com valores
        'sim' ou 'não' (strings).

    Retorna
    -------
    pd.DataFrame
        O mesmo DataFrame, com a coluna 'incluso_avaliacao' adicionada.
    """
    cols = list(criterios.keys())
    # Verifica se as colunas existem no DataFrame
    for col in cols:
        if col not in df.columns:
            raise ValueError(f"Coluna {col} não encontrada no DataFrame.")
    # Conta quantos 'true' por linha
    true_counts = df[cols].eq(True).sum(axis=1)
    # Define 'relevant' = 'true' se true_counts >= 3, senão 'false'
    df['incluso_avaliacao'] = (true_counts > 0).map({True: True, False: False})
    return df

In [ ]:
resultados_com_criterios_aplicados = apply_committee_criterios_exclusao(papers_criterios_exclusao, criterios_exclusao)

In [ ]:
print(resultados_com_criterios_aplicados[resultados_com_criterios_aplicados['incluso_avaliacao'] == True].shape)

## Critérios de inclusão

1. Forneceu feedback automatizados para os estudantes
2. Artigos primários
3. Publicado nos últimos quinze anos (abranger o “boom” da IA)
4. Estudos envolvendo aprendizes em qualquer nível educacional (fundamental, médio, superior, formação corporativa) que utilizem um AVA
5. Implementações de inteligência artificial (machine learning, NLP, agentes conversacionais, sistemas especialistas, etc.) voltadas à geração de feedback automatizado
6. Ambientes virtuais de aprendizagem (Moodle, Canvas, Blackboard, Google Classroom, Open edX, entre outros)
7. Trabalhos que relatem pelo menos um dos seguintes resultados:

    7.1. Qualidade / utilidade do feedback

    7.2. Impacto no desempenho acadêmico ou engajamento

    7.3. Satisfação dos estudantes ou docentes
    
    7.4. Métricas de eficiência do sistema (tempo, custo, escalabilidade).

8. Estudos empíricos (experimentos controlados, quase‑experimentos, estudos de caso, design‑based research) e relatos de desenvolvimento avaliados (artigos de conference/journal com validação)
9. Publicações revisadas por pares: artigos de periódicos, capítulos de livro, anais de conferências; teses e dissertações defendidas


In [ ]:
bibfile.data = resultados_com_relevancia.reset_index()
bibfile.data['relevant'] = bibfile.data['relevant'].astype(bool)
bibfile.data['relevant'] = bibfile.data['relevant'].replace({True: 'yes', False: 'no'})
bibfile.data['relevant'] = bibfile.data['relevant'].astype(str)
bibfile.data = bibfile.data.drop(columns=['relevant_llama3', 'relevant_gemma3', 'relevant_phi4'])
bibfile.word_cloud_plot(entry = 'abs', size_x = 15, size_y = 10, wordsn = 500, rmv_custom_words = [])

In [ ]:
if 'publication year' in bibfile.data.columns:
    bibfile.data['publication year'] = bibfile.data['publication year'].astype(str)
    bibfile.data['publication year'] = pd.to_numeric(bibfile.data['publication year'], errors='coerce')
    bibfile.data = bibfile.data.dropna(subset=['publication year'])
    bibfile.data = bibfile.data.rename(columns={'publication year': 'year'})
print(bibfile.data.columns)
bibfile.plot_evolution_year(view             = 'notebook',
                            stop_words       = ['en'],
                            rmv_custom_words = [],
                            key              = 'abs',
                            topn             = 10,
                            txt_font_size    = 12,
                            start            = 2010,
                            end              = 2025)

In [ ]:
bibfile.sankey_diagram(view = 'notebook', entry = ['aut', 'cout', 'inst', 'lan'], topn = 3)

In [ ]:
bibfile.network_sim(view = 'notebook', sim_type = 'cocit', node_size = 10, node_labels = True, cut_coup = 0.3, cut_cocit = 5)